# Azure OpenAI Structured output

This notebook shows how to use the structured output capability with the Azure OpenAI service.This differs from the older JSON mode feature, which could generate valid JSON but couldn't strictly enforce adherence to the provided schema.

You can read more about Structured outputs on OpenAI's blog: https://openai.com/index/introducing-structured-outputs-in-the-api/

**NOTE:** only gpt-4o version: 2024-08-06 supports structured outputs.

# Setup
First, we install the necessary dependencies and import the libraries we will be using.

In [ ]:
! pip install openai==1.44.0 

In [ ]:
from typing import Dict, List, Optional
from pydantic import BaseModel, Field
from openai import AzureOpenAI
import json

# Azure OpenAI Client Setup


**Authentication using API key**

Login Azure Portal and you can find this key in "Keys and Endpoints" under "Resource Management" in the Azure Portal.

Here initializes an AzureOpenAI client using endpoint, API key, and version.

Replace **AZURE_OPENAI_ENDPOINT** and **AZURE_OPENAI_API_KEY** with your key.


In [ ]:
client = AzureOpenAI(
    azure_endpoint = "AZURE_OPENAI_ENDPOINT",
    api_key="AZURE_OPENAI_API_KEY",
    api_version="2024-07-01-preview"
)

# Pydantic Structure

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Customer(BaseModel):
    id: str
    name: str
    email: Optional[str]
    phone: Optional[str]

class Product(BaseModel):
    id: str
    name: str
    price: float
    stock: int

class ShoppingCart(BaseModel):
    customer_details: Customer
    products: List[Product] = Field(
        description="List of products added to the shopping cart. Each entry must include product details as per the `Product` class."
    )

Here converting the ShoppingCart model into JSON schema.you can use any format.


In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "manage_shopping_cart",
            "description": "Manage shopping cart operations, including adding, removing, or viewing products in a customer's shopping cart",
            "parameters": ShoppingCart.model_json_schema(),
        }
    }
]


# Invoking the API with Messages and Tools

In [ ]:
shopping_cart_data = """ 
Customer Details: ID C123, Name John Doe, Email johndoe@example.com, Phone +91-9876543210. 
Products: 
1. ID P001, Name Laptop, Price 999.99, Stock 5. 
2. ID P002, Name Wireless Mouse, Price 25.99, Stock 50. 
"""


Inside the client.chat.completions.create, with tools=tools, the Pydantic structure is executed strictly, ensuring that the data adheres to the predefined schema without 

deviation. This strict enforcement guarantees that any operations related to the shopping cart, such as adding, removing, or viewing products, follow the exact structure 

and validation rules defined by the Pydantic models.


In [ ]:
messages = [
    {"role": "system", "content": "You are an AI assistant specialized in managing shopping cart operations. Use the supplied tools to assist the user with adding, removing, or viewing products in their shopping cart."},
    {"role": "user", "content": f"""Can you help me manage my shopping cart? Here are the details: {shopping_cart_data}"""}
]

response = client.chat.completions.create(
    model="example-model",  # replace with your actual model deployment name
    messages=messages,
    tools=tools
)


# Handling the Response

Prints Response data in a formatted JSON output.

In [ ]:
if response.choices and response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    function_args = tool_call.function.arguments

    response_data = json.loads(function_args)

    print(json.dumps(response_data, indent=2))
else:
    print("No function call was made in the response.")
